# Introduction

## Final Project Submission

***
- Student Name: Adam Marianacci
- Student Pace: Flex
- Scheduled project review date/time: TBD
- Instructor Name: Mark Barbour

# Business Understanding

# Data Understanding

# Data Preperation

In [ ]:
# Importing the necessary libraries

%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import numpy as np
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3
import matplotlib.pyplot as plt
import string
import re

In [ ]:
# Loading the data, and looking at the shape of the df

corpus = pd.read_csv('data/twitter_sentiment.csv', encoding='latin1')
corpus.shape

In [ ]:
# previewing the dataframe
corpus.head()

In [ ]:
corpus.info()

In [ ]:
corpus.describe()

In [ ]:
# renaming the 'emotion_in_tweet_is_directed_at' column to 'emotion_directed_at'
# # renaming the 'is_there_an_emotion_directed_at_a_brand_or_product' column to 'emotion_type'
corpus.rename(columns={'emotion_in_tweet_is_directed_at': 'emotion_directed_at',
                      'is_there_an_emotion_directed_at_a_brand_or_product': 'emotion_type'}, 
              inplace=True)

In [ ]:
# Inspecting the values in 'emotion_directed_at'
corpus['emotion_directed_at'].value_counts()


In [ ]:
# Replace values to group under 'Apple'
corpus['emotion_directed_at'].replace({'iPad': 'Apple',
                                       'iPad or iPhone App': 'Apple',
                                       'iPhone': 'Apple',
                                       'Other Apple product or service': 'Apple'},
                                      inplace=True)


In [ ]:
# Replace values to group under 'Google'
corpus['emotion_directed_at'].replace({'Other Google product or service': 'Google',
                                      'Android App': 'Google',
                                      'Android': 'Google'},
                                     inplace=True)

In [ ]:
# Inspecting the values in 'emotion_directed_at' to see if everything worked.
corpus['emotion_directed_at'].value_counts()

Apple has roughly 1/3 more emotions directed atthan Google

In [ ]:
corpus['emotion_type'].value_counts()

In [ ]:
# Filtering the df to exclude specified categories so it only contains positive or negative
clear_emotion_corpus = corpus[(corpus['emotion_type'] != "No emotion toward brand or product") 
                         & (corpus['emotion_type'] != "I can't tell")]

In [ ]:
clear_emotion_corpus.head()

In [ ]:
# Checking the data types and looking for missing values
clear_emotion_corpus.info()

We seem to have some missing values in the 'emotion_directed_at' column

In [ ]:
# Bring in stopwords

sw = stopwords.words('english')

In [ ]:
def doc_preparer(doc, stop_words=sw):
    ...
    
    :param doc: 
    :return: 
        
    ...
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:[a-z]+)?')")
    doc = regex_token.tokenizer(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in sw]

# Modeling

# Evaluation

# Conclusion

## Limitations

## Recommendations

## Next Steps